## MiBici Guadalajara, México

### Feature Selection: Forward Selection

A continuación realizaremos el feature selection de nuestro data set usando el método "Forward Selection"

In [107]:
!pip install mlxtend

In [108]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
import sys
import joblib
from IPython.display import display, HTML
from sklearn.metrics import r2_score
sys.modules['sklearn.externals.joblib'] = joblib

In [109]:
# loading aaour dataset
raw_df = pd.read_csv('../challenge_three/datasets/mibici_2021_2023.csv')

# removing outliers
# calculating upper and lower limit to avoid outliers
upper_limit_age = raw_df['age'].mean() + 2* raw_df['age'].std() # Right from the mean
lower_limit_age = raw_df['age'].mean() - 2* raw_df['age'].std() # Left from the mean
# trip duartion outliers
upper_limit_trip_duration = raw_df['trip_duration_hours'].mean() + 2* raw_df['trip_duration_hours'].std() # Right from the mean
lower_limit_trip_duration = raw_df['trip_duration_hours'].mean() - 2* raw_df['trip_duration_hours'].std() # Left from the mean

# filtering data to avoid outliers in age column
df = raw_df.copy()
df = df[(df['age']>lower_limit_age) & (df['age']<upper_limit_age)]
df = df[(df_cleaned['trip_duration_hours']>lower_limit_trip_duration) & (df['trip_duration_hours']<upper_limit_trip_duration)]
df = df.dropna()

In [110]:
#target
# df["gender"] = 1 if df["gender"] == "M" else 0
df['gender'] = df['gender'].replace(['M', 'F'], [1, 2])
X = df.drop(
    columns=[
        "trip_duration_minutes",
        "trip_id",
        "birthday",
        'start_trip', 
        'end_trip',
        'trip_duration_hours',
        'origin_name',
        'origin_obcn',
        'origin_location',
        'origin_status',
        'destination_name',
        'destination_obcn',
        'destination_location',
        'destination_status'
    ], 
    axis=1
)
y = df["trip_duration_minutes"]

display(X.head())
display(y)

,user_id,gender,age,year,origin_id,origin_latitude,origin_longitude,destination_id,destination_latitude,destination_longitude
0,70123,1,56.0,2023,64,20.673072,-103.365055,141,20.65381,-103.40134
1,1515654,2,29.0,2023,50,20.674721,-103.358548,141,20.65381,-103.40134
2,306600,1,32.0,2023,131,20.666914,-103.402512,141,20.65381,-103.40134
3,1107631,1,27.0,2023,131,20.666914,-103.402512,141,20.65381,-103.40134
4,1744633,1,29.0,2023,313,20.663707,-103.417817,141,20.65381,-103.40134


0          22.0
1          36.0
2           9.0
3          13.0
4          10.0
           ... 
1167884     8.0
1167885     8.0
1167888     7.0
1167889    13.0
1167890     7.0
Name: trip_duration_minutes, Length: 1146009, dtype: float64

In [111]:
# Sequential Forward Selection(sfs)
sfs = SFS(
    LinearRegression(),
    k_features=10,
    forward=True,
    floating=False,
    scoring = 'r2',
    cv = 0
)

In [112]:
sfs.fit(X, y)

SequentialFeatureSelector(cv=0, estimator=LinearRegression(),
                          k_features=(10, 10), scoring='r2')

In [113]:
# final list of features
sfs.k_feature_names_

('user_id',
 'gender',
 'age',
 'year',
 'origin_id',
 'origin_latitude',
 'origin_longitude',
 'destination_id',
 'destination_latitude',
 'destination_longitude')

## Entrenamiento de modelo de regresión lineal

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df[list(sfs.k_feature_names_)]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)
from sklearn.linear_model import LinearRegression
# crear el modelo
lr = LinearRegression()
# Ajustar el modelo con X_train y y_train
lr.fit(X_train,y_train)
# PRedecir con X_test
predictions = lr.predict(X_test)

print(predictions)

[10.33141384 11.02660015 11.68563378 ...  9.64865839 11.24952665
 11.15896754]


## Métricas de validación del modelo

In [117]:
from sklearn.metrics import mean_absolute_error
print("MAE",mean_absolute_error(y_test,y_pred))

MAE 5.369274355754961


In [118]:
from sklearn.metrics import mean_squared_error
print("MSE",mean_squared_error(y_test,y_pred))

MSE 47.50180098750773


In [119]:
print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred)))

RMSE 6.892155032173009


In [120]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,y_pred)
print("R2", r2)

R2 0.019339900491486595


## Conclusiones

Como podemos observar en la seleccion de características, hay varias columnas que pueden no ser importantes para el modelo de regresión lineal, para poder entender cual conjunto de características es mejor para el modelo tendríamos que evaluar varias combinaciones de características así como el número de ellas y evaluar el resultado de las métricas del conjunto de características entrenado.